In [8]:
import mlflow
from mlflow.client import MlflowClient
import pandas as pd
import os
from overfit.env_settings import settings


In [9]:
client = MlflowClient(tracking_uri=settings.MLFLOW_TRACKING_URI)


In [74]:
experiment_names = ["IMAGENET1K-4-50", "IMAGENET1K-5-50", "IMAGENET1K-6-75"]
experiment = client.get_experiment_by_name(experiment_names[2])


In [82]:
assert experiment is not None
runs = client.search_runs([experiment.experiment_id])


In [94]:
pimps = []
for run in runs:
    N = int(run.data.params["Frames"])
    src_acc_top1 = run.data.metrics["Source Accumulated Top-1 Count"]
    tgt_acc_top1 = run.data.metrics["Target Accumulated Top-1 Count"]
    if src_acc_top1 < 20:
        continue
    pimp = 100 * (tgt_acc_top1 - src_acc_top1) / (src_acc_top1)
    pimps.append(pimp)
    if (pimp == -100):
        print(run.data.params["Filename"])
        print(src_acc_top1, tgt_acc_top1)

pimps = pd.Series(pimps)
pimps.describe()

count    182.000000
mean      15.206735
std       18.931874
min      -15.000000
25%        3.791727
50%       10.840188
75%       21.573359
max      114.285714
dtype: float64

In [99]:
src_errors = []
tgt_errors = []
for run in runs:
    N = int(run.data.params["Frames"])
    src_acc_top1 = run.data.metrics["Source Accumulated Top-1 Count"]
    tgt_acc_top1 = run.data.metrics["Target Accumulated Top-1 Count"]
    src_errors.append((N - src_acc_top1)/N)
    tgt_errors.append((N - tgt_acc_top1)/N)

df = pd.DataFrame({"Source Errors": src_errors, "Target Errors": tgt_errors})
df.describe()

,Source Errors,Target Errors
count,514.000000,514.000000
mean,0.735470,0.694715
std,0.189946,0.220975
min,0.030303,0.015152
25%,0.636364,0.590909
50%,0.772727,0.727273
75%,0.878788,0.863636
max,1.000000,1.000000


In [49]:
for run in runs[:100]:
    N = int(run.data.params["Frames"])
    src_acc_top1 = run.data.metrics["Source Accumulated Correct Count"]
    tgt_acc_top1 = run.data.metrics["Target Accumulated Correct Count"]
    if src_acc_top1 > tgt_acc_top1:
        print(run.data.params["Filename"])
        print(src_acc_top1, tgt_acc_top1)


data/video/994-stinkhorn-5-65.mp4
30.0 29.0
data/video/984-rapeseed-5-65.mp4
33.0 32.0
data/video/979-valley-5-65.mp4
16.0 1.0
data/video/976-promontory-5-65.mp4
8.0 6.0
data/video/970-alp-5-65.mp4
29.0 16.0
data/video/968-cup-5-65.mp4
4.0 2.0
data/video/964-potpie-5-65.mp4
41.0 40.0
data/video/962-meat_loaf-5-65.mp4
14.0 13.0
data/video/961-dough-5-65.mp4
7.0 4.0
data/video/941-acorn_squash-5-65.mp4
9.0 7.0
data/video/935-mashed_potato-5-65.mp4
16.0 13.0
data/video/917-comic_book-5-65.mp4
16.0 15.0
data/video/910-wooden_spoon-5-65.mp4
10.0 8.0
